In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data_path = '../input/eplresults18.csv' # Path to data file
data = pd.read_csv(data_path) 
data.head(15)

In [ ]:
data.columns

In [ ]:
data.describe()

In [ ]:
data['FTR'].value_counts().plot(kind='bar')

In [ ]:
data['FTHG'].value_counts().plot(kind='bar')

In [ ]:
data['FTAG'].value_counts().plot(kind='bar')

In [ ]:
#Let's view the distribution of the data, where is it possible to find groups?
#We are using boxplots of all the columns except the first (cust_id which is a string)
for col in data.columns[2:2]:
    data[col].plot(kind='box')
    plt.title('Box Plot for '+col)
    plt.show()

In [ ]:
def create_label_encoder_dict(df):
    from sklearn.preprocessing import LabelEncoder
    
    label_encoder_dict = {}
    for column in df.columns:
        # Only create encoder for categorical data types
        if not np.issubdtype(df[column].dtype, np.number) and column not in ['Season','Date','HomeTeam','AwayTeam','Referee','FTHG','FTAG','HTAG','HS''AS''HF''HC''AC''HY'] :
            label_encoder_dict[column]= LabelEncoder().fit(df[column].astype(str))
    return label_encoder_dict

In [ ]:
label_encoders = create_label_encoder_dict(data)
print("Encoded Values for each Label")
print("="*32)
for column in label_encoders:
    print("="*32)
    print((column, label_encoders[column].classes_ ))
    print(pd.DataFrame([range(0,len(label_encoders[column].classes_))], columns=label_encoders[column].classes_, index=['Encoded Values']  ).T)

In [ ]:
# Apply each encoder to the data set to obtain transformed values
data2 = data.copy() # create copy of initial data set
for column in data2.columns:
    if column in label_encoders:
        data2[column] = label_encoders[column].transform(data2[column])

print("Transformed data set")
print("="*32)
data2

In [ ]:
cluster_data = data[['HC','AC','FTHG','FTAG']]
cluster_data.head()

In [ ]:
cluster_data.plot(kind='scatter',x='FTAG',y='FTHG')

In [ ]:
cluster_data.plot(kind='scatter',x='FTHG',y='AC')

In [ ]:
cluster_data.plot(kind='scatter',x='FTAG',y='HC')

In [ ]:
data.columns

In [ ]:
#data.dropna()
#data.fillna(0)

In [ ]:
# Is there any missing data
missing_data_results = cluster_data.isnull().sum()
print(missing_data_results)

# perform imputation with median values
# not require since none missing
#cluster_data = cluster_data.fillna( data.median() )

In [ ]:
data

In [ ]:
#retrieve just the values for all columns except customer id
data_values = cluster_data.iloc[ :, :].values
data_values

In [ ]:
#import KMeans algorithm
from sklearn.cluster import KMeans

In [ ]:
# Use the Elbow method to find a good number of clusters using WCSS (within-cluster sums of squares)
wcss = []
for i in range( 1,15 ):
    kmeans = KMeans(n_clusters=i, init="k-means++", n_init=10, max_iter=300) 
    kmeans.fit_predict( data_values )
    wcss.append( kmeans.inertia_ )
    
plt.plot( wcss, 'ro-', label="WCSS")
plt.title("Computing WCSS for KMeans++")
plt.xlabel("Number of clusters")
plt.ylabel("WCSS")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3, init="k-means++", n_init=10, max_iter=300) 
cluster_data["cluster"] = kmeans.fit_predict( data_values )
cluster_data

In [ ]:
cluster_data['cluster'].value_counts()

In [ ]:
cluster_data['cluster'].value_counts().plot(kind='bar',title='Distribution of Corners across teams')


In [ ]:
sns.pairplot( cluster_data, hue="cluster")

In [ ]:
grouped_cluster_data = cluster_data.groupby('cluster')
grouped_cluster_data

In [ ]:
grouped_cluster_data.describe()